# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [1]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [2]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/TR_SYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [3]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

vectorstore = Chroma.from_documents(documents=data, embedding=embeddings)

In [4]:
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Retrieve and generate answers using relevant FAQs
retriever = vectorstore.as_retriever()

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer from the provided context, just say that your training materials don't include this information, don't try to make up an answer.
Keep the answer as concise as possible.

{context}

Question: {question}

Helpful Answer:"""

prompt_template_TR = """Aşağıdaki bilgilerle kulanarak soran soruyu cevaplayın. Eğer verilen bilgileriyle soruyu cevaplamak mümkün değilse, verilmiş bilgilerde sorunun cevabı bulunmadğını açıklayın.

{context}

Soru: {question}

Yardımcı cevabı:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template_TR)

llm = Bedrock(model_id="amazon.titan-text-express-v1", model_kwargs={"maxTokenCount": 4000})

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [6]:
# Define a function to extract unique URLs used in the retrieved source materials.
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string

# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke("Çocuklarımı okula kaydetmek için neler lazım?")
print(response["answer"])
print(f"Sources: {extract_unique_urls(response)}")

 Çocuğunuz okul kaydı için öncelikle bulunduğunuz yerdeki İlçe Milli Eğitim Müdürlüğü’ne gidip size ve çocuğunuza ait 99 ile başlayan yabancı kimlik numarasını içeren Geçici Koruma Kimlik Belgeleri’ni sunmanız gerekmektedir. Okul kayıtları ikamet adresiniz esas alınarak yapıldığından, İlçe Nüfus Müdürlüğü’nden alacağınız, ikamet ettiğiniz adresi gösteren belgeyi de okul kayıt işlemleri esnasında yanınızda bulundurunuz. İlçe Milli Eğitim Müdürlüğü, evinize en yakın ve kapasitesi itibariyle müsait olan bir okula çocuğunuzun kaydını gerçekleştirecektir.

Çocuğunuzu kayıt ettireceğiniz sınıfa göre, sizlerden başka belgeler de talep edilebilmektedir. Bunların başında, çocuğunuzun daha önce aldığı eğitimi gösteren belgeler gelmektedir. Eğer çocuğunuzun daha önceki eğitim seviyesini gösteren belgeniz yoksa, çocuğunuzun hangi sınıfa yerleştirileceğine dair gerekli değerlendirmeyi İl veya İlçe Milli Eğitim Müdürlüğü bünyesinde tesis edilen İlçe Eğitim Komisyonu yapacaktır. İlçe Eğitim Komisyonu